In [61]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix



In [62]:
file_dataset = "/Users/gouravkatha/Desktop/flyfin/flyfin_interview_sample.csv"
result_path = "/Users/gouravkatha/Desktop/flyfin/result.csv"

In [63]:
df = pd.read_csv(file_dataset,  dtype=str)
print(df.shape)
df.head(10)

(3644, 5)


,id,original_amount,txn_date,plaid_category,plaid_dirty_name
0,1312,3.5,2019-11-20,['Food and Drink'],PepsiCo
1,97452,5.09,2020-11-04,"['Travel', 'Gas Stations']",FOOD SMAR 2901 W ARMIT CHICAGO IL 11/04
2,39582,9.39,2019-05-01,"['Food and Drink', 'Restaurants', 'Fast Food']",Chick-fil-A
3,97707,10.2,2020-09-14,"['Travel', 'Gas Stations']",FOOD SMAR 2901 W ARMIT CHICAGO IL 09/13
4,90662,53.4,2020-11-06,"['Shops', 'Pets']",CHEWY.COM POS FL US XX4330
5,79034,16.6,2020-10-02,"['Service', 'Food and Beverage']",DOORDASH*DUNKIN WWW.DOORDASH.
6,98910,10.0,2019-10-15,"['Travel', 'Public Transportation Services']",VENTRA MOBILE IL 10/14
7,85098,14.95,2020-07-23,"['Shops', 'Bookstores']",Audible
8,48099,9.35,2020-11-01,"['Shops', 'Beauty Products']",Sephora
9,64956,2.0,2020-12-10,"['Travel', 'Parking']",POS DEBIT NYCDOT PARKING METERS


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [65]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sparse_dot_topn import awesome_cossim_topn


class Matcher():
    def __init__(self, df, columns_to_group, match_threshold=0.75, ngram_remove=r'[,-./]', ngram_length=3):
        self.df = df
        self.group_lookup = {}
        self._column = self._get_column(columns_to_group)
        self._match_threshold = match_threshold
        self._ngram_remove = ngram_remove
        self._ngram_length = ngram_length

    def _get_column(self, columns_to_group):
        if ''.join(columns_to_group) in self.df.columns:
            return ''.join(columns_to_group)
        else:
            self.df['Grouper'] = self.df[columns_to_group.pop(0)].astype(str).str.cat(self.df[columns_to_group].astype(str))
            return 'Grouper'

    def _ngrams_analyzer(self, string):
        string = re.sub(self._ngram_remove, r'', string)
        ngrams = zip(*[string[i:] for i in range(self._ngram_length)])
        return [''.join(ngram) for ngram in ngrams]

    def _get_tf_idf_matrix(self, vals):
        vectorizer = TfidfVectorizer(analyzer=self._ngrams_analyzer)
        return vectorizer.fit_transform(vals)

    def _get_cosine_matrix(self, vals):
        tf_idf_matrix = self._get_tf_idf_matrix(vals)
        return awesome_cossim_topn(tf_idf_matrix, tf_idf_matrix.transpose(), vals.size, self._match_threshold)

    def _find_group(self, y, x):
        if y in self.group_lookup:
            return self.group_lookup[y]
        elif x in self.group_lookup:
            return self.group_lookup[x]
        else:
            return None

    def _add_vals_to_lookup(self, group, y, x):
        self.group_lookup[y] = group
        self.group_lookup[x] = group

    def _add_pair_to_lookup(self, row, col):
        group = self._find_group(row, col)
        if group is not None:
            self._add_vals_to_lookup(group, row, col)
        else:
            self._add_vals_to_lookup(row, row, col)

    def set_ngram_remove(self, ngram_remove):
        self._ngram_remove = ngram_remove

    def set_ngram_length(self, ngram_length):
        self._ngram_length = ngram_length

    def set_match_threshold(self, match_threshold):
        self._match_threshold = match_threshold

    def build_group_lookup(self):
        vals = self.df[self._column].unique().astype('U')

        print('Building the TF-IDF, Cosine & Coord matrices...')
        coord_matrix = self._get_cosine_matrix(vals).tocoo()

        print('Building the group lookup...')
        for row, col in zip(coord_matrix.row, coord_matrix.col):
            if row != col:
                self._add_pair_to_lookup(vals[row], vals[col])

    def add_grouped_column_to_data(self, column_name='Group'):
        print('Adding grouped columns to data frame...')
        self.df[column_name] = self.df[self._column].map(self.group_lookup).fillna(self.df[self._column])

    def run(self, column_name='Group'):
        self.build_group_lookup()
        self.add_grouped_column_to_data(column_name)
        return self.df.drop(columns=['Grouper']) if 'Grouper' in self.df.columns else self.df


In [66]:
def read_csv(csv_path, columns_to_group, match_threshold=0.75, ngram_remove=r'[,-./]', ngram_length=3):
    return Matcher(pd.read_csv(csv_path), columns_to_group, match_threshold, ngram_remove, ngram_length)

In [67]:
matcher = read_csv(file_dataset, ['plaid_category', 'plaid_dirty_name'], match_threshold=0.8, ngram_length=5)

In [68]:
result=matcher.run()
result.head(10)

Building the TF-IDF, Cosine & Coord matrices...
Building the group lookup...
Adding grouped columns to data frame...


,id,original_amount,txn_date,plaid_category,plaid_dirty_name,Group
0,1312,3.50,2019-11-20,['Food and Drink'],PepsiCo,['Food and Drink']PepsiCo
1,97452,5.09,2020-11-04,"['Travel', 'Gas Stations']",FOOD SMAR 2901 W ARMIT CHICAGO IL 11/04,"['Travel', 'Gas Stations']FOOD SMAR 2901 W ARMIT CHICAGO IL 11/04"
2,39582,9.39,2019-05-01,"['Food and Drink', 'Restaurants', 'Fast Food']",Chick-fil-A,"['Food and Drink', 'Restaurants', 'Fast Food']Chick-fil-A"
3,97707,10.20,2020-09-14,"['Travel', 'Gas Stations']",FOOD SMAR 2901 W ARMIT CHICAGO IL 09/13,"['Travel', 'Gas Stations']FOOD SMAR 2901 W ARMIT CHICAGO IL 11/04"
4,90662,53.40,2020-11-06,"['Shops', 'Pets']",CHEWY.COM POS FL US XX4330,"['Shops', 'Pets']CHEWY.COM POS FL US XX4330"
5,79034,16.60,2020-10-02,"['Service', 'Food and Beverage']",DOORDASH*DUNKIN WWW.DOORDASH.,"['Service', 'Food and Beverage']DOORDASH*DUNKIN WWW.DOORDASH."
6,98910,10.00,2019-10-15,"['Travel', 'Public Transportation Services']",VENTRA MOBILE IL 10/14,"['Travel', 'Public Transportation Services']VENTRA MOBILE IL 10/14"
7,85098,14.95,2020-07-23,"['Shops', 'Bookstores']",Audible,"['Shops', 'Bookstores']Audible"
8,48099,9.35,2020-11-01,"['Shops', 'Beauty Products']",Sephora,"['Shops', 'Beauty Products']Sephora"
9,64956,2.00,2020-12-10,"['Travel', 'Parking']",POS DEBIT NYCDOT PARKING METERS,"['Travel', 'Parking']POS DEBIT NYCDOT PARKING METERS"


In [69]:
print("Category and Frequency is as follows")
frequency_df = result["Group"].value_counts()
frequency_df.head(10)

Category and Frequency is as follows


['Food and Drink', 'Restaurants', 'Fast Food']Chick-fil-A             428
['Travel', 'Public Transportation Services']VENTRA MOBILE IL 10/14    372
['Shops', 'Beauty Products']Sephora                                   141
['Shops', 'Pharmacies']Duane Reade                                    133
['Shops', 'Digital Purchase']GLOBALPOK GOLD COINS TA XBIEX 06/20      129
['Shops', 'Supermarkets and Groceries']ADEN FOOD MARKET               117
['Shops', 'Discount Stores']Dollar General                            116
['Travel', 'Parking']NYCDOT PARKING METERS LONG IS CITY NY 07/31      112
['Service', 'Insurance']State Farm                                    112
['Transfer', 'Debit']ALLY CASHBACK                                    103
Name: Group, dtype: int64

In [70]:
def export_csv(df, export_path=None):
    df.to_csv(result_path)

In [71]:
export_csv(frequency_df, result_path)